In [1]:
# -------- Setup and Installations --------
import os
# Install packages
#!apt-get remove -y openjdk-21-jdk

!add-apt-repository ppa:linuxuprising/java -y
!apt-get update
!apt-get install openjdk-17-jdk -y
#!apt-get install -y gradle

# Download and install Gradle 7.3
#!apt-get remove -y gradle
!wget https://services.gradle.org/distributions/gradle-7.3-bin.zip
!unzip -d /opt/gradle gradle-7.3-bin.zip
!rm gradle-7.3-bin.zip
os.environ["PATH"] = "/opt/gradle/gradle-7.3/bin:" + os.environ["PATH"]


# Install Python packages
!pip install music21 gradio

# -------- Environment Configuration --------

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:" + os.environ['PATH']
os.environ['TESSDATA_PREFIX'] = '/root/tessdata/'

# Set up Tesseract OCR data
!mkdir -p ~/tessdata
!wget https://github.com/tesseract-ocr/tessdata/raw/main/eng.traineddata -P ~/tessdata/

# -------- Imports --------
from music21 import converter
import gradio as gr

!java --version
!gradle --version

Repository: 'deb https://ppa.launchpadcontent.net/linuxuprising/java/ubuntu/ jammy main'
Description:
Oracle Java 11 (LTS) and 17 (LTS) installer for Ubuntu (21.10, 21.04, 20.04, 18.04, 16.04 and 14.04), Pop!_OS, Linux Mint and Debian.

Java binaries are not hosted in this PPA due to licensing. The packages in this PPA download and install Oracle Java, so a working Internet connection is required.

The packages in this PPA are based on the WebUpd8 Oracle Java PPA packages: https://launchpad.net/~webupd8team/+archive/ubuntu/java

Installation instructions (with some tips), feedback, suggestions, bug reports etc.:

Oracle Java 11: https://www.linuxuprising.com/2019/06/new-oracle-java-11-installer-for-ubuntu.html
Oracle Java 17: https://www.linuxuprising.com/2021/09/how-to-install-oracle-java-17-lts-on.html

Important notice regarding Oracle Java 11 and 16: the Oracle JDK license has changed starting April 16, 2019. The new license permits certain uses, such as personal use and developmen

In [2]:
!java --version
!gradle --version
# Remove any existing audiveris directory to avoid conflicts
#!rm -rf /content/audiveris

# Clone Audiveris repository
!git clone https://github.com/Audiveris/audiveris.git
# Change directory to audiveris
%cd /content/audiveris
!git -C audiveris checkout development
!git -C audiveris pull --all
!pwd

!./gradlew clean build

openjdk 17.0.13 2024-10-15
OpenJDK Runtime Environment (build 17.0.13+11-Ubuntu-2ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.13+11-Ubuntu-2ubuntu122.04, mixed mode, sharing)

------------------------------------------------------------
Gradle 7.3
------------------------------------------------------------

Build time:   2021-11-09 20:40:36 UTC
Revision:     96754b8c44399658178a768ac764d727c2addb37

Kotlin:       1.5.31
Groovy:       3.0.9
Ant:          Apache Ant(TM) version 1.10.11 compiled on July 10 2021
JVM:          17.0.13 (Ubuntu 17.0.13+11-Ubuntu-2ubuntu122.04)
OS:           Linux 6.1.85+ amd64

Cloning into 'audiveris'...
remote: Enumerating objects: 116198, done.
remote: Counting objects: 100% (18785/18785), done.
remote: Compressing objects: 100% (5430/5430), done.
remote: Total 116198 (delta 12997), reused 18192 (delta 12475), pack-reused 97413 (from 1)
Receiving objects: 100% (116198/116198), 320.85 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (92317/92317), don

In [ ]:
import shutil
import time

def process_pdf_to_midi(pdf_file):
    # Create a unique subdirectory in /tmp based on the PDF file name and timestamp
    unique_folder_name = f"{os.path.splitext(os.path.basename(pdf_file.name))[0]}_{int(time.time())}"
    output_dir = os.path.join('/tmp', unique_folder_name)
    os.makedirs(output_dir, exist_ok=True)
    print(f"Created output directory: {output_dir}")

    # Define paths for PDF, MusicXML, and MIDI files within the unique folder
    pdf_file_path = os.path.join(output_dir, os.path.basename(pdf_file.name))
    xml_file = os.path.join(output_dir, os.path.splitext(os.path.basename(pdf_file.name))[0] + '.mxl')
    midi_output = os.path.join(output_dir, 'output_file.mid')

    # Copy the uploaded PDF to the unique folder
    shutil.copy(pdf_file.name, pdf_file_path)
    print(f"Copied PDF file to: {pdf_file_path}")

    # Run Audiveris to generate MusicXML file
    audiveris_command = f'./gradlew run --args="-batch -export -output {output_dir} {pdf_file_path}"'
    print(f"Running Audiveris command: {audiveris_command}")
    os.system(audiveris_command)

    # Verify if MusicXML file was generated
    if not os.path.exists(xml_file):
        print("Error: MusicXML file not found.")
        return "Error: MusicXML file not found."  # Return an error string if XML is missing
    print(f"Generated MusicXML file at: {xml_file}")

    # Convert MusicXML to MIDI
    score = converter.parse(xml_file)
    score.write('midi', fp=midi_output)
    print(f"MIDI file generated successfully at: {midi_output}")

    return midi_output

# Gradio interface function
def gradio_interface(pdf_file):
    midi_file_path = process_pdf_to_midi(pdf_file)
    if "Error" in midi_file_path:
        return midi_file_path  # Return error if processing failed
    return midi_file_path  # Return MIDI path if successful

# Launch Gradio interface
gr.Interface(
    fn=gradio_interface,
    inputs=gr.File(label="Upload Sheet Music PDF"),
    outputs=gr.File(label="Download MIDI File"),
    title="AI Piano Tutor - Convert Sheet Music PDF to MIDI",
    description="Upload a sheet music PDF to convert it into a MIDI file."
).launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://23297f68e8d4ea6a91.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
